In [ ]:
#restart session after running these, you wont have to run them again
%pip install xformers
%pip install -i https://pypi.org/simple/ bitsandbytes
%pip install accelerate
%pip install peft trl
!pip install "unsloth[colab-new] @ git+https://GitHub.com/unslothai/unsloth.git"

In [1]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
import numpy as np
import pandas as pd


In [2]:
from unsloth import FastLanguageModel

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


In [ ]:
!huggingface-cli login

In [3]:
#dont load this model
'''
model = AutoModelForCausalLM.from_pretrained(
    "meta-llama/Meta-Llama-3-8B-Instruct",
    load_in_4bit=True,
    #attn_implementation="flash_attention_2",
)
tokenizer = AutoTokenizer.from_pretrained("meta-llama/Meta-Llama-3-8B")
'''

The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.
`low_cpu_mem_usage` was None, now set to True since model is quantized.


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [4]:
device = 'cuda:0'

In [5]:
df_test = pd.read_csv('test_stock_data (1).csv')
df_train = pd.read_csv('train_stock_data (1).csv')
df_val = pd.read_csv('val_stock_data (1).csv')

In [14]:
df_test.head()

,Company,title,description,Date,Up/Down
0,Apple,"Apple Proved That AI Is a Feature, Not a Product",Other tech companies want to sell you chatbots...,2024-06-13,Up
1,Apple,"Apple Won't Pay for ChatGPT, Will You?",Apple and OpenAI announced a landmark partners...,2024-06-13,Up
2,Apple,The AI upgrade cycle is here,The new Apple Intelligence features coming to ...,2024-06-13,Up
3,Apple,Apple CEO Tim Cook on the Magic Mouse: 'Uhhhhh...,Apple has invented some phenomenal products ov...,2024-06-13,Up
4,Apple,"My favorite iOS 18, iPadOS 18 and watchOS 11 f...",There was so much Apple had to cram into its W...,2024-06-13,Up


In [9]:
df_test['Company'].unique()

array(['Apple', 'Amazon', 'Tesla', 'Google', 'Disney', 'Netflix',
       'Nvidia'], dtype=object)

In [8]:
df_test.shape

(2655, 5)

In [ ]:
df_train

In [10]:
df_train.groupby(['Company','Date']).count()

title  description  Up/Down
Company Date                                   
Amazon  2024-05-21     64           64       64
        2024-05-22     52           52       52
        2024-05-23     63           63       63
        2024-05-24     58           58       58
        2024-05-28     53           53       53
...                   ...          ...      ...
Tesla   2024-06-12     49           49       49
        2024-06-13     60           60       60
        2024-06-14     58           58       58
        2024-06-17     54           54       54
        2024-06-18     54           54       54

[140 rows x 3 columns]

In [12]:
df_val.groupby(['Company','Date']).count()

title  description  Up/Down
Company Date                                   
Amazon  2024-05-21     15           15       15
        2024-05-22     21           21       21
        2024-05-23     15           15       15
        2024-05-24     19           19       19
        2024-05-28     21           21       21
...                   ...          ...      ...
Tesla   2024-06-12     18           18       18
        2024-06-13     13           13       13
        2024-06-14     13           13       13
        2024-06-17     19           19       19
        2024-06-18     20           20       20

[140 rows x 3 columns]

In [ ]:
results = []
for comp in df_test['Company'].unique():
    df_comp = df_test[df_test['Company'] == comp]
    for date in df_test['Date'].unique():
        f5 = df_comp[df_comp['Date'] == date].iloc[0:15]
        text_prompt = f"""<|begin_of_text|><|start_header_id|>system<|end_header_id|>

        You are a helpful AI assistant for stock market tips and recommendations<|eot_id|><|start_header_id|>user<|end_header_id|>

        I am going to provide you with some news blurbs about {comp} from today, and a multiple choice question at the end. Please memorize the news, then answer the question. Here are the blurbs:"""
        for i in range(15):
            text_prompt += f"1) {f5['title'].iloc[i]} : {f5['description'].iloc[i]}\n"
        text_prompt += f"\nNow here is the question: Based on your understanding of the news, determine whether {comp}'s stock will go up or down by open tomorrow. Please respond with either (A) - UP or (B) - DOWN.<|eot_id|><|start_header_id|>assistant<|end_header_id|>The correct answer is \n"
        input_ids = tokenizer(text_prompt, return_tensors='pt').input_ids
        input_ids = input_ids.to(device)
        out = model.generate(input_ids, max_new_tokens=2, do_sample=False, temperature=0)
        pred = tokenizer.decode(out[0])[-10:]
        print(comp, date, pred)
        results.append([comp, date, pred])

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:515: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:520: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_m

Apple 2024-05-21 er is 
(B)


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Apple 2024-05-22 er is 
(B)


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


...

Nvidia 2024-06-13 er is 
(A)


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Nvidia 2024-06-14 er is 
(B)


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Nvidia 2024-06-17 er is 
(B)
Nvidia 2024-06-18 wer is 
A)


In [ ]:
results = [
    'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B',
    'B', 'B', 'B', 'B', 'B', 'B', 'B', 'A', 'B', 'B',
    'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'A', 'B',
    'B', 'B', 'B', 'B', 'B', 'B', 'B', 'A', 'B', 'A',
    'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B',
    'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B',
    'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B',
    'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B',
    'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B',
    'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B',
    'A', 'A', 'B', 'B', 'B', 'B', 'A', 'B', 'A', 'B',
    'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B',
    'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B',
    'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B',
    'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B',
    'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'B',
    'B', 'A', 'A', 'B', 'A', 'A', 'A', 'B', 'B', 'A'
]

In [ ]:
labels = []
for comp in df['Company'].unique():
    df_comp = df[df['Company'] == comp]
    for date in df_comp['Date'].unique():

        label = df_comp[df_comp['Date'] == date]['Up/Down'].iloc[0]
        labels.append(label)
print(labels)


['Up', 'Down', 'Down', 'Up', 'Up', 'Up', 'Up', 'Up', 'Up', 'Up', 'Up', 'Down', 'Up', 'Down', 'Up', 'Up', 'Up', 'Down', 'Up', 'Down', 'Down', 'Down', 'Down', 'Down', 'Up', 'Down', 'Down', 'Down', 'Up', 'Up', 'Up', 'Up', 'Down', 'Up', 'Up', 'Down', 'Down', 'Down', 'Up', 'Down', 'Up', 'Down', 'Down', 'Up', 'Down', 'Down', 'Up', 'Down', 'Down', 'Down', 'Up', 'Up', 'Down', 'Down', 'Down', 'Up', 'Up', 'Down', 'Up', 'Down', 'Up', 'Down', 'Down', 'Up', 'Up', 'Down', 'Down', 'Up', 'Up', 'Up', 'Up', 'Up', 'Down', 'Up', 'Up', 'Up', 'Down', 'Up', 'Up', 'Down', 'Up', 'Up', 'Down', 'Up', 'Up', 'Down', 'Up', 'Up', 'Down', 'Up', 'Down', 'Down', 'Up', 'Up', 'Down', 'Down', 'Down', 'Down', 'Up', 'Down', 'Up', 'Down', 'Down', 'Up', 'Up', 'Up', 'Down', 'Down', 'Down', 'Down', 'Up', 'Down', 'Down', 'Up', 'Up', 'Up', 'Up', 'Up', 'Up', 'Up', 'Up', 'Down', 'Up', 'Up', 'Up', 'Up', 'Down', 'Down', 'Up', 'Up', 'Up', 'Down', 'Down', 'Up', 'Down', 'Up', 'Up', 'Up', 'Down', 'Up']


In [ ]:
labels = [1 if x == 'Up' else 0 for x in labels]

In [ ]:
labels

[1,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 0,
 1,
 1,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 1,
 1,
 0,
 1,
 0,
 1,
 0,
 0,
 1,
 1,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 0,
 1,
 1,
 0,
 1,
 1,
 0,
 1,
 1,
 0,
 1,
 1,
 0,
 1,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 0,
 0,
 1,
 1,
 1,
 0,
 0,
 1,
 0,
 1,
 1,
 1,
 0,
 1]

In [ ]:
results

[['Apple', '2024-05-21', 'er is \n(B)'],
 ['Apple', '2024-05-22', 'er is \n(B)'],
 ['Apple', '2024-05-23', 'er is \n(B)'],
 ['Apple', '2024-05-24', 'er is \n(B)'],
 ['Apple', '2024-05-28', 'er is \n(B)'],
 ['Apple', '2024-05-29', 'er is \n(B)'],
 ['Apple', '2024-05-30', 'er is \n(B)'],
 ['Apple', '2024-05-31', 'er is \n(B)'],
 ['Apple', '2024-06-03', 'er is \n(B)'],
 ['Apple', '2024-06-04', 'er is \n(B)'],
 ['Apple', '2024-06-05', 'er is \n(B)'],
 ['Apple', '2024-06-06', 'er is \n(B)'],
 ['Apple', '2024-06-07', 'er is \n(B)'],
 ['Apple', '2024-06-10', 'er is \n(A)'],
 ['Apple', '2024-06-11', 'er is \n(B)'],
 ['Apple', '2024-06-12', 'er is \n(B)'],
 ['Apple', '2024-06-13', 'er is \n(A)'],
 ['Apple', '2024-06-14', 'er is \n(B)'],
 ['Apple', '2024-06-17', 'er is \n(B)'],
 ['Apple', '2024-06-18', 'er is \n(B)'],
 ['Amazon', '2024-05-21', 'er is \n(B)'],
 ['Amazon', '2024-05-22', 'er is \n(B)'],
 ['Amazon', '2024-05-23', 'er is \n(B)'],
 ['Amazon', '2024-05-24', 'er is \n(B)'],
 ['Amazon', 

In [ ]:
res = [1 if 'A' in x[2] else 0 for x in results]

In [ ]:
res

[0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 1,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 1]

In [ ]:
len(y_true)

140

In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Replace these with your actual predictions and true labels
y_true = labels  # True labels
y_pred = res  # Predicted labels

# Calculate metrics
accuracy = accuracy_score(y_true, y_pred)
precision = precision_score(y_true, y_pred)
recall = recall_score(y_true, y_pred)
f1 = f1_score(y_true, y_pred)

# Print the results
print(f'Accuracy: {accuracy:.2f}')
print(f'Precision: {precision:.2f}')
print(f'Recall: {recall:.2f}')
print(f'F1 Score: {f1:.2f}')


Accuracy: 0.41
Precision: 0.43
Recall: 0.17
F1 Score: 0.24


In [ ]:
torch.cuda.empty_cache()

## Fine-tuning with qLORA

In [6]:
from torch.utils.data import Dataset, DataLoader

In [7]:
training_prompts = []
training_labels = []
for comp in df_train['Company'].unique():
    df_comp = df_train[df_train['Company'] == comp]
    for date in df_train['Date'].unique():
        f5 = df_comp[df_comp['Date'] == date].iloc[0:15]
        text_prompt = f"""<|begin_of_text|><|start_header_id|>system<|end_header_id|>

        You are a helpful AI assistant for stock market tips and recommendations<|eot_id|><|start_header_id|>user<|end_header_id|>

        I am going to provide you with some news blurbs about {comp} from today, and a multiple choice question at the end. Please memorize the news, then answer the question. Here are the blurbs:"""
        for i in range(15):
            text_prompt += f"1) {f5['title'].iloc[i]} : {f5['description'].iloc[i]}\n"
        text_prompt += f"\nNow here is the question: Based on your understanding of the news, determine whether {comp}'s stock will go up or down by open tomorrow. Please respond with either (A) - UP or (B) - DOWN.<|eot_id|>" #<|start_header_id|>assistant<|end_header_id|>The correct answer is \n"
        training_prompts.append(text_prompt)
        lbl = df_comp[df_comp['Date'] == date]['Up/Down'].iloc[0]
        training_labels.append(lbl)

In [20]:
len(training_prompts)

84

In [21]:
training_labels = ['(A)' if i == 'Up' else '(B)' for i in training_labels]
training_labels

['(A)',
 '(B)',
 '(B)',
 '(A)',
 '(A)',
 '(A)',
 '(A)',
 '(A)',
 '(A)',
 '(A)',
 '(A)',
 '(B)',
 '(B)',
 '(B)',
 '(B)',
 '(B)',
 '(A)',
 '(B)',
 '(B)',
 '(B)',
 '(A)',
 '(A)',
 '(A)',
 '(A)',
 '(A)',
 '(B)',
 '(B)',
 '(A)',
 '(B)',
 '(B)',
 '(A)',
 '(B)',
 '(B)',
 '(B)',
 '(A)',
 '(A)',
 '(A)',
 '(B)',
 '(B)',
 '(A)',
 '(A)',
 '(B)',
 '(B)',
 '(A)',
 '(A)',
 '(A)',
 '(A)',
 '(A)',
 '(A)',
 '(A)',
 '(B)',
 '(A)',
 '(A)',
 '(B)',
 '(A)',
 '(A)',
 '(B)',
 '(A)',
 '(B)',
 '(B)',
 '(A)',
 '(B)',
 '(B)',
 '(A)',
 '(A)',
 '(A)',
 '(B)',
 '(B)',
 '(B)',
 '(B)',
 '(A)',
 '(B)',
 '(A)',
 '(B)',
 '(A)',
 '(A)',
 '(A)',
 '(A)',
 '(B)',
 '(B)',
 '(A)',
 '(A)',
 '(A)',
 '(B)']

In [22]:
from datasets import Dataset, DatasetDict


In [23]:
data_dict = {
    'prompt': training_prompts,
    'output': training_labels
}

# Create a Dataset from the dictionary
dataset = Dataset.from_dict(data_dict)

# If you want to create a DatasetDict with train, validation, and test splits
dataset_dict = DatasetDict({
    'train': dataset,
    #'validation': dataset,  # Replace with actual validation set
    #'test': dataset  # Replace with actual test set
})

In [28]:
from peft import LoraConfig, get_peft_model

In [27]:
from trl import SFTTrainer

from transformers import (
    DataCollatorForLanguageModeling, TrainingArguments, Trainer
)

In [31]:
# Defining the configuration for the base model, LoRA and training
config = {
    "hugging_face_username":"sanil41",
    "model_config": {
        "base_model":'unsloth/llama-3-8b-Instruct-bnb-4bit', # The base model
        "finetuned_model":"llama-3-8b-Instruct-bnb-4bit-finance", # The fine-tuned model
        "max_seq_length": 8000, # The maximum sequence length
        "dtype":torch.float16, # The data type
        "load_in_4bit": True, # Load the model in 4-bit
    },
    "lora_config": {
      "r": 16, # The number of LoRA layers 8, 16, 32, 64
      "target_modules": ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj"], # The target modules
      "lora_alpha":16, # The alpha value for LoRA
      "lora_dropout":0, # The dropout value for LoRA
      "bias":"none", # The bias for LoRA
      "use_gradient_checkpointing":True, # Use gradient checkpointing
      "use_rslora":False, # Use RSLora
      "use_dora":False, # Use DoRa
      "loftq_config":None # The LoFTQ configuration
    },
    "training_config": {
        "per_device_train_batch_size": 2, # The batch size
        "gradient_accumulation_steps": 4, # The gradient accumulation steps
        "warmup_steps": 5, # The warmup steps
        "max_steps":0, # The maximum steps (0 if the epochs are defined)
        "num_train_epochs": 1, # The number of training epochs(0 if the maximum steps are defined)
        "learning_rate": 2e-4, # The learning rate
        "fp16": not torch.cuda.is_bf16_supported(), # The fp16
        "bf16": torch.cuda.is_bf16_supported(), # The bf16
        "logging_steps": 1, # The logging steps
        "optim" :"adamw_8bit", # The optimizer
        "weight_decay" : 0.01,  # The weight decay
        "lr_scheduler_type": "linear", # The learning rate scheduler
        "seed" : 42, # The seed
        "output_dir" : "outputs", # The output directory
    }
}
'''
    "training_dataset":{
        "name":"Shekswess/medical_llama3_instruct_dataset_short", # The dataset name(huggingface/datasets)
        "split":"train", # The dataset split
        "input_field":"prompt", # The input field
    },
'''

'\n    "training_dataset":{\n        "name":"Shekswess/medical_llama3_instruct_dataset_short", # The dataset name(huggingface/datasets)\n        "split":"train", # The dataset split\n        "input_field":"prompt", # The input field\n    },\n'

In [25]:
dataset_dict

DatasetDict({
    train: Dataset({
        features: ['prompt', 'output'],
        num_rows: 84
    })
})

In [26]:
# Loading the model and the tokinizer for the model
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = config.get("model_config").get("base_model"),
    max_seq_length = config.get("model_config").get("max_seq_length"),
    dtype = config.get("model_config").get("dtype"),
    load_in_4bit = config.get("model_config").get("load_in_4bit"),
)

# Setup for QLoRA/LoRA peft of the base model
model = FastLanguageModel.get_peft_model(
    model,
    r = config.get("lora_config").get("r"),
    target_modules = config.get("lora_config").get("target_modules"),
    lora_alpha = config.get("lora_config").get("lora_alpha"),
    lora_dropout = config.get("lora_config").get("lora_dropout"),
    bias = config.get("lora_config").get("bias"),
    use_gradient_checkpointing = config.get("lora_config").get("use_gradient_checkpointing"),
    random_state = 42,
    use_rslora = config.get("lora_config").get("use_rslora"),
    use_dora = config.get("lora_config").get("use_dora"),
    loftq_config = config.get("lora_config").get("loftq_config"),
)

# Loading the training dataset
#dataset_train = load_dataset(config.get("training_dataset").get("name"), split = config.get("training_dataset").get("split"))



==((====))==  Unsloth: Fast Llama patching release 2024.6
   \\   /|    GPU: Tesla T4. Max memory: 14.748 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.3.0+cu121. CUDA = 7.5. CUDA Toolkit = 12.1.
\        /    Bfloat16 = FALSE. Xformers = 0.0.26.post1. FA = False.
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth


model.safetensors:  14%|#4        | 807M/5.70G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/131 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/51.1k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/459 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Unsloth 2024.6 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


NameError: name 'SFTTrainer' is not defined

In [35]:
# Setting up the trainer for the model
trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset_dict['train'],
    dataset_text_field = 'prompt',
    max_seq_length = config.get("model_config").get("max_seq_length"),
    dataset_num_proc = 2,
    packing = False,
    args = TrainingArguments(
        per_device_train_batch_size = config.get("training_config").get("per_device_train_batch_size"),
        gradient_accumulation_steps = config.get("training_config").get("gradient_accumulation_steps"),
        warmup_steps = config.get("training_config").get("warmup_steps"),
        max_steps = config.get("training_config").get("max_steps"),
        num_train_epochs= 4,
        learning_rate = config.get("training_config").get("learning_rate"),
        fp16 = config.get("training_config").get("fp16"),
        bf16 = config.get("training_config").get("bf16"),
        logging_steps = config.get("training_config").get("logging_steps"),
        optim = config.get("training_config").get("optim"),
        weight_decay = config.get("training_config").get("weight_decay"),
        lr_scheduler_type = config.get("training_config").get("lr_scheduler_type"),
        seed = 42,
        output_dir = config.get("training_config").get("output_dir"),
    ),
)

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1965: FutureWarning: `--push_to_hub_token` is deprecated and will be removed in version 5 of 🤗 Transformers. Use `--hub_token` instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:269: UserWarning: You passed a `max_seq_length` argument to the SFTTrainer, the value you passed will override the one in the `SFTConfig`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:283: UserWarning: You passed a `dataset_num_proc` argument to the SFTTrainer, the value you passed will override the one in the `SFTConfig`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:307: UserWarning: You passed a `dataset_text_field` argument to the SFTTrainer, the value you passed will override the one in the `SFTConfig`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/multiprocess/popen_fork.py:66: RuntimeWarning: os.fork() was c

Map (num_proc=2):   0%|          | 0/84 [00:00<?, ? examples/s]

In [33]:
gpu_statistics = torch.cuda.get_device_properties(0)
reserved_memory = round(torch.cuda.max_memory_reserved() / 1024**3, 2)
max_memory = round(gpu_statistics.total_memory / 1024**3, 2)
print(f"Reserved Memory: {reserved_memory}GB")
print(f"Max Memory: {max_memory}GB")

Reserved Memory: 5.59GB
Max Memory: 14.75GB


In [ ]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 84 | Num Epochs = 4
O^O/ \_/ \    Batch size per device = 2 | Gradient Accumulation steps = 4
\        /    Total batch size = 8 | Total steps = 40
 "-____-"     Number of trainable parameters = 41,943,040


Step,Training Loss
1,2.275800
2,2.284500
3,2.344100
4,2.270400
5,2.179500
6,1.995500
7,2.014600
8,2.043200
9,2.017600


In [23]:
torch.cuda.empty_cache()